Extracting User's Google Calendar

In [1]:
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from google.auth.transport.requests import Request
import googleapiclient.discovery
import datetime


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import re
import ollama
import pandas as pd
from ollama import Client
from ollama import chat
import methods
import formats
import json

google_calendar = methods.get_google_calendar_service() #get the google calendar service
calendar = methods.extractCalendar(google_calendar) #extract the calendar from the service
now = methods.current_time()

Logging Previous Conversation

In [4]:
def log_message(role, message):
    filename = f"conversation_log.txt"
    with open(filename, "a") as file:
        timestamp = datetime.datetime.now().isoformat()
        file.write(f"{timestamp} | {role}: {message}\n")

In [14]:
def get_past_conversation():
    filename = f"conversation_log.txt"
    try:
        with open(filename, "r") as file:
            return file.read()
    except FileNotFoundError:
        return ""

Get User's Query

In [5]:
#past_conversation = get_past_conversation()
context = f"Here's my calendar for the week, store it for your reference. Today's date and time is: {now}\n{calendar}"
instruction = """\nAs my virtual assistant, you are an expert with managing the calendar.
You can create, move, check, list, update, delete, and handle recurring events based on the user's requests.
When the user doesn't provide enough information, you can ask for more details, up to 3 times. Otherwise use your best judgement.
When the user wants to create an event or move an event and if the user hasn't specified when they want it scheduled, then refer to the calendar and use your best judgement to schedule the event. Keep in mind when the user would normally eat or sleep. Assume default duration for any event as 1 hour.
Check for any conflicts before scheduling an event.
Always ask for confirmation before making any changes to the calendar, if the user doesn't agree, then generate another suitable time slot.
When you are done, please let the user know that the task has been completed, and don't ask any more questions. If you aren't done, end with a question mark.
Please perform the necessary actions using the Google Calendar API.\n"""

# initial setup
initial_prompt = context + instruction
# past conversation confuses llama2

In [6]:
user_request = input()
combined_prompt = initial_prompt + "\nUser request: " + user_request


In [7]:
def llama2(client, messages):
    response = ollama.chat(model='llama2', messages=messages)
    print(response)
    return response['message']['content']

messages = [
    {
        "role": "system",
        "content": initial_prompt
    },
    {
        "role": "user",
        "content": user_request
    }
]
log_message("user", user_request)

In [37]:
ollama.pull("llama2")

{'status': 'success'}

In [8]:
finalized_response = False
client = Client(host='http://localhost:11434')
#establishing feedback loop to chat with user
while not finalized_response:
    response = llama2(client, messages)
    print(response)
    log_message("assistant", response)
    messages.append({
        "role": "assistant",
        "content": response
    })
    if re.compile(r'\?').search(response):
        finalized_response = False
        user_feedback = input()
        messages.append({
            "role": "user",
            "content": user_feedback
        })
        log_message("user", user_feedback)
    else:
        finalized_response = True


{'model': 'llama2', 'created_at': '2024-04-03T21:15:59.71137Z', 'message': {'role': 'assistant', 'content': "Great, I'm happy to help! Based on your calendar information, it looks like there are no conflicts for a jog tomorrow morning. How about we schedule the jog for 7:00 AM tomorrow? Would that work for you?"}, 'done': True, 'total_duration': 99401285479, 'load_duration': 3229377756, 'prompt_eval_count': 960, 'prompt_eval_duration': 78630384000, 'eval_count': 53, 'eval_duration': 17535850000}
Great, I'm happy to help! Based on your calendar information, it looks like there are no conflicts for a jog tomorrow morning. How about we schedule the jog for 7:00 AM tomorrow? Would that work for you?
{'model': 'llama2', 'created_at': '2024-04-03T21:32:38.814676Z', 'message': {'role': 'assistant', 'content': 'Great! I\'ve created an event named "Jog" with a start time of 7:00 AM tomorrow (April 4th) and an end time of 8:00 AM. The event id is "545 Project Meeting". Is that correct?\n\nPlease

In [9]:
#get Llama2 to output the function calls I need to make to the calendar API
instruction2 = """\nBased on the user's request you've just processed, please output the appropriate function call(s) to interact with the Google Calendar API. Format your response as a single string containing one or more function calls separated by semicolons. Do not include explanations or additional text.
All dates should be in 'YYYY-MM-DD' format, and times should be in 24-hour format 'HH:MM'. Additionally, for functions requiring a datetime range (such as 'checkSchedule' and 'ListEvents'), the datetime should be in ISO 8601 format 'YYYY-MM-DDTHH:MM:SSZ'.
Available function templates to include in the response string:
- createEvent(description, date:default=today, time, duration:default=1)
- moveEvent(event_id, new_date, new_time)
- updateEvent(event_id, new_description, new_duration, new_location)
- deleteEvent(event_id)
- createRecurringEvent(description, date, time, duration, frequency, until)
- deleteRecurringEvent(event_id)
- updateRecurringEvent(event_id, new_description, new_duration, new_location)

Make sure to fill in the required information, such as event names, IDs, dates, times, etc., based on the user's request context. Replace placeholders like 'today' or 'tomorrow' with specific dates in the format YYYY-MM-DD. If more details are needed from the user for a function call, use your best judgment to complete the function with the information available.

The following are examples:
Example input: 'Schedule a team meeting for Friday afternoon at 3pm'
Example output: 'createEvent("Team Meeting", "2024-03-12", "15:00", 1)'

Now, based on the user's request you just processed and the event that you have now modified / created, please generate the necessary function call and ONLY output the function with filled in information based on the user's information you just processed. Based on my request, format the action that you provided accurately based on the above descriptions. Make sure each parameter is formatted correctly."""

In [10]:
messages.append({
    "role": "system",
    "content": instruction2
})
print('messages: ', messages)
function_call = llama2(client, messages)
print(function_call)
#Example of function call: createEvent("Meeting with John", "2022-12-31", "10:00 AM", 2)

messages:  [{'role': 'system', 'content': "Here's my calendar for the week, store it for your reference. Today's date and time is: 2024-04-03T17:14:06.199272Z\n{'name': '545 Project Meeting', 'datetime_start': '2024-04-04T14:00:00-04:00', 'datetime_end': '2024-04-04T14:30:00-04:00', 'id': '4k8s6unih4r88130q6cccmv2tq_20240404T180000Z'}\n{'name': 'roomie brunch', 'datetime_start': '2024-04-05T10:30:00-04:00', 'datetime_end': '2024-04-05T12:00:00-04:00', 'id': '6d851ccms5kfjmfgn9rj3cg515'}\n{'name': 'Flight to Atlanta', 'datetime_start': '2024-04-05T14:30:00-04:00', 'datetime_end': '2024-04-05T15:30:00-04:00', 'id': '0pcp0a45mek9n4fi0i71g94bm0'}\n{'name': 'Flight back to Detroit ', 'datetime_start': '2024-04-07T16:00:00-04:00', 'datetime_end': '2024-04-07T17:00:00-04:00', 'id': '3ekshci1a26b43t9hj0vpm6k67'}\n{'name': '545 Project Meeting', 'datetime_start': '2024-04-08T16:00:00-04:00', 'datetime_end': '2024-04-08T18:00:00-04:00', 'id': '33pmlrcvmobnp4oc9t4d64ib2o_20240408T200000Z'}\n{'nam

In [20]:
function_calls = [line.strip() for line in function_call.split(';')]
print(function_calls)

# Define a dictionary mapping function names to their corresponding Python functions
functions = {
    "createEvent": methods.createEvent,
    "moveEvent": methods.moveEvent,
    "checkSchedule": methods.checkSchedule,
}

#TODO: works for createEvent, gotta change other functions in methods file to accept arguments as strings and convert them to appropriate data types
# Iterate through the function calls
for call in function_calls:
    # Split the string to extract function name and arguments
    if call == '': continue
    parts = call.split('(')
    function_name = parts[0]

    arguments = parts[1].rstrip(')').split(', ')
    

    # Call the corresponding function with the provided arguments
    if function_name in functions:
        functions[function_name](*arguments)
    else:
        print(f"Function '{function_name}' not found.")

['createEvent("Jog", "2024-04-04", "7:00", 1)', '']
Create event called
The data and time is  2024-04-04 / 7:00
UTC start time:  2024-04-04 11:00:00+00:00 UTC end time:  2024-04-04 12:00:00+00:00
Event created: Jog on 2024-04-04 at 7:00 for duration of 1 hours.
